In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb
import pandas as pd

In [2]:
label_encoder = LabelEncoder()

df = pd.read_csv('Processed Data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 92 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rowid               9564 non-null   int64  
 1   koi_disposition     9564 non-null   object 
 2   koi_score           9564 non-null   float64
 3   koi_fpflag_nt       9564 non-null   int64  
 4   koi_fpflag_ss       9564 non-null   int64  
 5   koi_fpflag_co       9564 non-null   int64  
 6   koi_fpflag_ec       9564 non-null   int64  
 7   koi_period          9564 non-null   float64
 8   koi_time0bk         9564 non-null   float64
 9   koi_time0           9564 non-null   float64
 10  koi_impact          9564 non-null   float64
 11  koi_impact_err1     9564 non-null   float64
 12  koi_impact_err2     9564 non-null   float64
 13  koi_duration        9564 non-null   float64
 14  koi_duration_err1   9564 non-null   float64
 15  koi_duration_err2   9564 non-null   float64
 16  koi_de

In [3]:
df['koi_disposition_encoded'] = label_encoder.fit_transform(df['koi_disposition'])

print("Original classes:", df['koi_disposition'].unique())
print("Encoded classes:", df['koi_disposition_encoded'].unique())
print("Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

Original classes: ['CONFIRMED' 'CANDIDATE' 'FALSE POSITIVE']
Encoded classes: [1 0 2]
Mapping: {'CANDIDATE': np.int64(0), 'CONFIRMED': np.int64(1), 'FALSE POSITIVE': np.int64(2)}


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['koi_disposition', 'koi_disposition_encoded'], axis=1), 
    df['koi_disposition_encoded'], 
    test_size=0.2, 
    random_state=42
)
clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2,device='cpu')

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)]
)

XGBoostError: [22:57:09] /workspace/src/common/common.cu:16: /workspace/src/common/cuda_dr_utils.cc: 30: cudaErrorUnknown: unknown error
Stack trace:
  [bt] (0) /home/ali/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x2a6e7c) [0x7fb0196a6e7c]
  [bt] (1) /home/ali/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xa5b61e) [0x7fb019e5b61e]
  [bt] (2) /home/ali/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x38303b) [0x7fb01978303b]
  [bt] (3) /home/ali/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x3840ac) [0x7fb0197840ac]
  [bt] (4) /lib/x86_64-linux-gnu/libc.so.6(+0x99ee8) [0x7fb0b8a99ee8]
  [bt] (5) /home/ali/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x382f68) [0x7fb019782f68]
  [bt] (6) /home/ali/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xa5cb55) [0x7fb019e5cb55]
  [bt] (7) /home/ali/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xa5bacd) [0x7fb019e5bacd]
  [

In [4]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Convert numeric predictions back to original labels for better interpretability
y_pred_labels = label_encoder.inverse_transform(y_pred.astype(int))
y_test_labels = label_encoder.inverse_transform(y_test.astype(int))


NameError: name 'clf' is not defined